# Data Cleaning

## Introduction

* This notebook goes through a necasssary step of cleaning the data before it is used for exploratory data analysis. 
* The input of this notebook is a training dataset in csv format sourced from Kaggle. https://www.kaggle.com/c/nlp-getting-started
* The output of this notebook is a csv file with clean and lemmatized text data. 

## Reading and Understanding the dataset

In [514]:
# Importing libraries
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re           # regular expression 
import string       # String Handling
import random       #For selecting random rows
import nltk         # Natural langauage processing toolkit
from nltk.stem import WordNetLemmatizer  #Used for Lemmatizing the text
from nltk.corpus import wordnet          #Used for POS tagging 
from nltk.corpus import stopwords        #Stopwords to be removed from text


Reading the training data into a dataframe using pandas and viewing the top 20 rows of data

In [515]:
train_df = pd.read_csv("train.csv")
train_df.head(20)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


First 20 rows in 'Keyword' and 'Location' column are NaNs. Now Checking if there are nulls in other columns of the dataframe.

In [516]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


Checking number of unique values in 'keywords' and 'location'.

In [517]:
len(train_df.keyword.unique())

222

In [518]:
len(train_df.location.unique())

3342

A glimpse at 50 rows from the 'locations' column selected randomly


In [519]:
random.sample(list((train_df.location.unique())),k=50)


['Wasington, DC',
 'Soul Somalia/Body Montreal',
 'london',
 'Jersey',
 'Denver, CO',
 'todaysbigstock.com',
 'hatena bookmark',
 'Your screen',
 '?Gangsta OC / MV RP; 18+.?',
 'N?? Y???.',
 'twitch.tv/naturalemblem26',
 'global',
 'USA',
 'Brackley Beach, PE, Canada',
 'Liverpool',
 'Email: Lovethterry@gmail.com',
 'Chicora ?? Oakland',
 'Dublin, Ireland',
 'Caracas, Venezuela.',
 'Spying on your thoughts',
 'Karachi, Pakistan',
 'NY',
 'Tallahassee Florida',
 'Galapa / AtlÌÁntico',
 'Atlanta, Georgia USA',
 'Los Angeles',
 'The Triskelion',
 'Oklahoma',
 'Oregon, USA',
 'Greenville,SC',
 'Republic of the Philippines',
 'Lansing, Michigan',
 'My old New England home',
 'Top secret bunker ',
 'USAoV',
 'teh internets',
 'ÌÏT: 33.209923,-87.545328',
 'Magnolia',
 'sÌ£o luis',
 'Perth, Western Australia',
 'Weyburn',
 'Torry Alvarez love forever ? ?',
 "L'Enfant Plaza Metro Station",
 'Kenosha, WI 53143',
 'Chicopee MA',
 'Canada Eh! ',
 'Hawaii, USA',
 'This Is Paradise. Relax. ',
 'Eng

A glimpse at 50 rows from the 'keywords' column selected randomly

In [520]:
random.sample(list((train_df.keyword.unique())),k=50)


['annihilated',
 'nuclear%20reactor',
 'detonation',
 'eyewitness',
 'displaced',
 'explode',
 'panic',
 'hijack',
 'derailment',
 'sinking',
 'survive',
 'thunderstorm',
 'drowned',
 'flattened',
 'cyclone',
 'attack',
 'evacuated',
 'exploded',
 'casualties',
 'hostages',
 'electrocute',
 'weapon',
 'lightning',
 'sirens',
 'wreck',
 'structural%20failure',
 'quarantined',
 'attacked',
 'deaths',
 'curfew',
 'hailstorm',
 'hijacking',
 'devastated',
 'crush',
 'evacuation',
 'collided',
 'destroyed',
 'screamed',
 'desolate',
 'emergency',
 'wounded',
 'body%20bag',
 'blizzard',
 'flooding',
 'explosion',
 'fatalities',
 'blew%20up',
 'obliterate',
 'inundated',
 'typhoon']

A glimpse at 30 rows from the 'text' column selected randomly

In [521]:
# Selecting 30 random from text column randomly
random.sample(list((train_df.text)),k=30)

['@LA_Siren Thanks for joining the foot. @VVorm',
 "Murfreesboro peeps- I'm hearing Walmart on S Rutherford is on lockdown with a hostage is that true or a rumor?",
 '1st Quality Insurance Group is #hiring Licensed Property &amp; Casualty Insurance Agent Produc http://t.co/VMJRtuVmh4 #jobs #Denver',
 'Obama Declares Disaster for Typhoon-Devastated Saipan: Obama signs disaster declaration for Northern Marians a... http://t.co/PC8BvufLFJ',
 'UD: Rescue (Structural Collapse) - Scott Road @ Ypres Road York (14 Trucks)',
 'remember that time goku gave life to a dead birb \nwhat the hell goku',
 '#EMERGENCY in Odai Bucharest Romania 600 Dogs Dying!They are so Hungry that they EAT EACH OTHER! http://t.co/pjigXPVPl0',
 "@pattonoswalt @FoxNews Wait I thought Fecal Hurricane was on SciFi? Maybe that was turdnado. I've been forgeting up a shit storm lately.",
 'Forest fires could delay @AbbyAirshow but officials say it could be a good thing http://t.co/Vxjcx8uKMd',
 'So grateful for all the suppo

## Observation

Some data incosistencies or redundant information found in the dataset are as follows

* Upper case and lower case at unexpected location
* Punctuations
* Numbers in text 
* Use of cities, states and Country names. (Granularity problem)
* Special characters such as \x89ÛÒ and \n
* Hyperklinks
* Tags in tweets




## Cleaning the data

The method below does the following to clean anomalies in 'location' column:

* Changes all the text to lower case
* Removes punctutations
* Removes texts with numbers
* Removes cities names if country/state names are mentioned. (High level granularity is maintained)

In [522]:
# Method to chaange text to lower case and remove punctionation
def cleaning_location(text):
    text = str(text)
    text = text.lower()    #lower case
    text = text.split(',')[-1:][0].strip() # Removing city names when country/state name is present
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #removing punctuations
    text = re.sub('\w*\d\w*', '', text) #removing text with number

    
    return text

# Applying the method to location column
train_df.location = train_df.location.apply(cleaning_location)

The method below does the following to clean anomalies in 'keyword' column:

* Changes all the text to lower case
* Removes punctutations
* replaces number with space (as %20 was found in middle of two words)

In [523]:
def cleaning_keyword(text):
    text = str(text)
    text = text.lower()    ##lower case
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  ##removing punctuations
    text = re.sub('(\d+)', ' ', text)   #replacing numbers with space
    return text

# Applying the method to keyword column
train_df.keyword = train_df.keyword.apply(cleaning_keyword)

The following method extracts the hashtags from the tweets. These hashtags play a vital role in interpreting the context of the tweet

In [524]:
def retreive_hashtags(text):
    hash_tag = ''
    hash_tag = re.sub('#','',' '.join(re.findall('(#[A-Za-z]+[A-Za-z0-9-_]+)', text)))  #retreiving hastags
    return hash_tag

# Applying the method to retreive hastags and add to the hastag column
train_df["Hasthags"] = ''
train_df.Hasthags= train_df.text.apply(retreive_hashtags)

The following method to clean anomalies in 'text' column does the following

* Changes all the text to lower case
* Removes words starting with @ to remove the tags and mentions example: @barackobama
* Adds a column with hashtag values
* Removes links
* Removes punctuation
* Removes words with numbers
* Removes special characters examples: \x89û,\x89ûó etc 
* Removes '\n' 


In [525]:
def cleaning_text(text):
    
    text = str(text)
    text = text.lower()    ##lower case
    
    text = re.sub(r'@[A-Za-z]+[A-Za-z0-9-_]+', '',text) #removing any word starting with @   \w
    text = re.sub(r'https|www|http\S+', '', text)  #removing any word starting with http
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  ##removing punctuations
    text = re.sub('\w*\d\w*', '', text)  #removing words with numbers
    text = re.sub(r'[^\x00-\x7F]+', '', text) # removing special characters
    text.replace("\n","")
    return text

The following method converts NLTK tags to wordnet tags which would be used to lemmatize the words in the following method

In [526]:
def wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

The following method is used to lemmatize the sentences in the following order:

* Clean the sentences by calling the cleaning_text method on each sentence.
* Tokenizing the sentence and generating a nltk POS tag for each word in the cleaned sentence.
* Converting the nltk POS tag to Wordnet POS tag by calling wordnet_tag method.
* Removes the stopwords
* Lemmatizes the tokens using the pos tags


In [527]:
lemmatizing = WordNetLemmatizer()

def lemmatize_sentence(sentence):
    sentence = cleaning_text(sentence) #Cleaning the sentence
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  #Tokenizing and tagging each word 
    wordnet_tagged = map(lambda x: (x[0], wordnet_tag(x[1])), nltk_tagged)  # Coverting the NLTK tags to wordnet tag
    
    #Lemmatizing the tagged tokens
    lemmatized_sentence = [] #empty list for lemmatized words
    for word, tag in wordnet_tagged:
        if word not in set(stopwords.words('english')):  #removing stopwords
            if tag is None:                   
                lemmatized_sentence.append(word) #adding the word as it is if POS tag missing
            else:        
                #else use the tag to lemmatize the token
                lemmatized_sentence.append(lemmatizing.lemmatize(word, tag))  ##lemmatizing the token using the POS tag
    return " ".join(lemmatized_sentence)



In [528]:
# Applying the lemmatizing method to Hashtags column 
train_df.Hasthags= train_df.Hasthags.apply(lemmatize_sentence)

# Applying the lemmatizing method to text column 
train_df.text= train_df.text.apply(lemmatize_sentence)

In [533]:
train_df.to_csv("Clean_train_data")

,id,keyword,location,text,target,Hasthags
5621,8019,refugees,nan,hmm nigerian refugee repatriate cameroon,1,
6274,8965,storm,mackem in bolton,every time buy bag life think ive get day leav...,0,
1731,2494,collided,nan,even bad look american constitutional system c...,0,
5051,7199,natural disaster,mì©xico df,ral send message condolence vietnam follow nat...,1,
4392,6244,hijacking,mongolia,hot funtenna hijack computer send data sound w...,1,hot prebreak best
4937,7032,mayhem,interplanetaryzone,best mind clear content observation day tcopcc...,0,
3804,5406,fire truck,nan,even sun article nno,0,
523,755,avalanche,ireland,little piece write avalanche design blog id ap...,0,
1423,2055,casualties,new yor,warfighting robot could reduce civilian casual...,1,ftsn ftsnnewsdesk
28,40,nan,nan,cooool,0,
